In [1]:
import ipywidgets as widgets
from IPython.display import display
from datetime import datetime

# Define the dropdown widgets for input
start_year = widgets.Dropdown(
    options=[str(year) for year in range(2000, 2025)],
    description='Start Year:'
)
start_month = widgets.Dropdown(
    options=[f"{month:02d}" for month in range(1, 13)],
    description='Start Month:'
)
start_day = widgets.Dropdown(
    options=[f"{day:02d}" for day in range(1, 32)],
    description='Start Day:'
)

end_year = widgets.Dropdown(
    options=[str(year) for year in range(2000, 2025)],
    description='End Year:'
)
end_month = widgets.Dropdown(
    options=[f"{month:02d}" for month in range(1, 13)],
    description='End Month:'
)
end_day = widgets.Dropdown(
    options=[f"{day:02d}" for day in range(1, 32)],
    description='End Day:'
)

aoi_input = widgets.Text(
    description="AOI (bbox):",
    placeholder="Enter bbox as minX,minY,maxX,maxY"
)

# Display the widgets
display(start_year, start_month, start_day, end_year, end_month, end_day, aoi_input)

# Function to retrieve input from widgets
def get_inputs():
    try:
        # Validate dates
        start_date = datetime(
            int(start_year.value), int(start_month.value), int(start_day.value)
        ).strftime('%Y-%m-%d')
        end_date = datetime(
            int(end_year.value), int(end_month.value), int(end_day.value)
        ).strftime('%Y-%m-%d')
        
        # Validate AOI input
        aoi = [float(coord) for coord in aoi_input.value.split(",")]
        if len(aoi) != 4:
            print("Invalid AOI format. Please enter as minX,minY,maxX,maxY.")
            return None, None, None
        
        return start_date, end_date, aoi
    except ValueError as e:
        print(f"Input error: {e}")
        return None, None, None

# List satellite collections with metadata
satellite_sources = {
    "Sentinel-2": {
        "Temporal Extent": "2015-Present",
        "Spatial Resolution": "10m (Visible, NIR), 20m (Red Edge, SWIR), 60m (Coastal, Aerosols)",
        "Return Time": "5 days",
        "Bands": "B02 (Blue), B03 (Green), B04 (Red), B08 (NIR), others"
    },
    "Landsat 8-9": {
        "Temporal Extent": "2013-Present",
        "Spatial Resolution": "30m",
        "Return Time": "16 days",
        "Bands": "B1 (Coastal), B2 (Blue), B3 (Green), B4 (Red), B5 (NIR), others"
    },

    "NAIP (US only)": {
        "Temporal Extent": "2010-Present (varies by state)",
        "Spatial Resolution": "0.3m-1m",
        "Return Time": "Every 2-3 years",
        "Bands": "R, G, B, NIR (varies)"
    },
}

# Print satellite source options and metadata
print("\nAvailable Satellite Data Sources for Vegetation Analysis:\n")
for source, metadata in satellite_sources.items():
    print(f"{source}:")
    for key, value in metadata.items():
        print(f"  {key}: {value}")
    print()

Dropdown(description='Start Year:', options=('2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '…

Dropdown(description='Start Month:', options=('01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11'…

Dropdown(description='Start Day:', options=('01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', …

Dropdown(description='End Year:', options=('2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '20…

Dropdown(description='End Month:', options=('01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', …

Dropdown(description='End Day:', options=('01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '1…

Text(value='', description='AOI (bbox):', placeholder='Enter bbox as minX,minY,maxX,maxY')


Available Satellite Data Sources for Vegetation Analysis:

Sentinel-2:
  Temporal Extent: 2015-Present
  Spatial Resolution: 10m (Visible, NIR), 20m (Red Edge, SWIR), 60m (Coastal, Aerosols)
  Return Time: 5 days
  Bands: B02 (Blue), B03 (Green), B04 (Red), B08 (NIR), others

Landsat 8-9:
  Temporal Extent: 2013-Present
  Spatial Resolution: 30m
  Return Time: 16 days
  Bands: B1 (Coastal), B2 (Blue), B3 (Green), B4 (Red), B5 (NIR), others

NAIP (US only):
  Temporal Extent: 2010-Present (varies by state)
  Spatial Resolution: 0.3m-1m
  Return Time: Every 2-3 years
  Bands: R, G, B, NIR (varies)



In [2]:
import ipywidgets as widgets
from shapely.geometry import box
import geopandas as gpd
import pystac_client

# Function to query satellite collections using the previously set widget values
def query_collections():
    # Retrieve the values from the widgets
    start_date = f"{start_year.value}-{start_month.value}-{start_day.value}"
    end_date = f"{end_year.value}-{end_month.value}-{end_day.value}"
    aoi = [float(coord) for coord in aoi_input.value.split(",")]

    # Validate AOI format
    if len(aoi) != 4:
        raise ValueError("Invalid AOI format. Please enter as minX,minY,maxX,maxY.")
    
    # Ensure AOI is a valid bounding box
    aoi_geometry = box(*aoi)
    bbox = aoi_geometry.bounds  # Get bounding box coordinates

    # MPC STAC API endpoint
    stac_url = "https://planetarycomputer.microsoft.com/api/stac/v1"
    catalog = pystac_client.Client.open(stac_url)

    # Satellite collections and their STAC IDs
    collections = {
        "Sentinel-2": "sentinel-2-l2a",
        "Landsat 8-9": "landsat-8-c2-l2",
        "NAIP (US only)": "naip",
    }

    # Results list to store collection data
    all_results = []

    # Query each collection
    for name, collection_id in collections.items():
        collection_box = widgets.VBox()  # Create a VBox for each collection's widget box
        try:
            search = catalog.search(
                collections=[collection_id],
                datetime=f"{start_date}/{end_date}",
                bbox=bbox
            )
            items = list(search.get_items())
            if items:
                # Count images
                num_images = len(items)
                # Calculate average cloud coverage
                cloud_coverages = [
                    float(item.properties.get("eo:cloud_cover", 0)) for item in items if "eo:cloud_cover" in item.properties
                ]
                avg_cloud_coverage = sum(cloud_coverages) / len(cloud_coverages) if cloud_coverages else "N/A"
                
                # Filter non-cloudy images (less than 5% cloud cover)
                non_cloudy_items = [
                    item for item in items 
                    if float(item.properties.get("eo:cloud_cover", 100)) < 5
                ]
                non_cloudy_images = len(non_cloudy_items)
                non_cloudy_dates = [
                    item.properties["datetime"][:10] for item in non_cloudy_items
                ]

                # Add the results as text to a text widget
                results_text = (
                    f"**Collection:** {name}\n"
                    f"**Number of Images:** {num_images}\n"
                    f"**Average Cloud Coverage:** {avg_cloud_coverage:.2f}\n"
                    f"**Non-Cloudy Images (<5% Cloud):** {non_cloudy_images}\n"
                )
                results_widget = widgets.Textarea(
                    value=results_text,
                    layout=widgets.Layout(width="90%", height="100px"),
                    disabled=True
                )

                # Create a scrollable list for low cloud cover dates
                low_cloud_dates_widget = widgets.VBox([widgets.Label(f"{date}") for date in non_cloudy_dates])
                
                # Create a scrollable container for the dates using a Box with layout overflow
                scrollable_widget = widgets.Box(
                    children=[low_cloud_dates_widget],
                    layout=widgets.Layout(
                        width="200%",
                        height="200px",
                        overflow="auto"  # This makes the content scrollable if too long
                    )
                )

                # Create an expandable panel for low cloud cover dates
                low_cloud_dates_widget = widgets.Accordion(children=[scrollable_widget])
                low_cloud_dates_widget.set_title(0, f"{name} Low Cloud Dates")

                # Add the results widget and expandable scrollable panel to the collection's VBox
                collection_box.children = [results_widget, low_cloud_dates_widget]

            else:
                # Handle no items case
                results_text = (
                    f"**Collection:** {name}\n"
                    f"**Number of Images:** 0\n"
                    f"**Average Cloud Coverage:** N/A\n"
                    f"**Non-Cloudy Images (<5% Cloud):** 0\n"
                )
                results_widget = widgets.Textarea(
                    value=results_text,
                    layout=widgets.Layout(width="90%", height="100px"),
                    disabled=True
                )

                # Add the results widget to the collection's VBox
                collection_box.children = [results_widget]

            # Add the collection's widget box to the main results list
            all_results.append(collection_box)

        except Exception as e:
            # Handle error case
            error_text = f"**Collection:** {name}\nError retrieving data: {e}\n"
            error_widget = widgets.Textarea(
                value=error_text,
                layout=widgets.Layout(width="90%", height="100px"),
                disabled=True
            )
            collection_box.children = [error_widget]
            all_results.append(collection_box)

    # Display all collection widgets
    display(*all_results)

# Call the query function immediately after defining it to load the results with widget inputs
query_collections()

/root/miniconda3/lib/python3.12/site-packages/pystac_client/item_search.py:888: FutureWarning: get_items() is deprecated, use items() instead
  warnings.warn(
